![The Hotel Monte Carlo](./resources/Hotel_Monte_Carlo_r3-01.png)

# Monte Carlo Methods and the Fundamental Unity of Statistical Tests

## Roadmap

In this chapter we will use what we know about coding to cheat at statistics using an important class of techniques called Monte Carlo methods. We will not be cheating in the sense of doing something unethical, but only in the sense of using what we know about python to answer statistical questions without doing very much math. In so doing, I hope that we will develop an intuitive sense for the types of questions statistical tests are trying to answer. At the outset, statistical methods can sometimes feel like a grab-bag of random formulas with little relation to one another: T-tests, Pearson and Spearman regression, ANOVA, etc. It is easy to miss the fundamental unity of the questions asked by these tests, and the approach they take to answer them. By using Monte Carlo methods, I hope we will begin to see that many statistical questions are often very simple and indeed nearly all follow a straightforward common pattern. 

Ultimately we will come full circle and discuss how even though we can answer a very wide array of statistical tests using Monte Carlo methods, traditional statistical methods can be much faster and more convenient when their assumptions hold true. 

## The Monte Carlo approach

Monte Carlo approaches are named after the Monte Carlo casino in Monaco. As the name suggests, these methods use random *simulation* or *permutation* to calculate statistical quantities. 

Here's a simple example of using simulation to solve a problem: say we want to know the probability of dealing three Aces in a hand of five cards drawn from a [standard 52 card deck](https://en.wikipedia.org/wiki/Standard_52-card_deck). If we have already taken a class or two in probability and statistics, we could calculate this probability mathematically. 

However, strictly speaking we don't need much math to work out an answer if we have a lot of time on our hands. In principle we could simply get a pack of cards, deal a hand many many times, and count how often you get three Aces. If you divide the number of times you get three Aces in a hand by how many hands you dealt, you would then get the **probability** or **p-value** of getting three A aces.

Of course, you probably didn't get into bioinformatics because you liked boring and repetitive tasks like dealing thousands of hands of cards. If we know how to code, we can instead simulate dealing a hand of cards in the computer, and automate the dealing of hands many times using a for loop.

We'll start our discussion of Monte Carlo methods solving this problem using python, then turn to more biological problems.

## Python functions useful for Monte Carlo tests

To be able to write Monte Carlo tests in python, you'll want to make sure you are comfortable and familiar with for loops, if statements, and writing your own functions. If not, I recommend going back and reviewing or learning these topics.

In addition, the functions in the [random](https://docs.python.org/3/library/random.html) module are often used when writing Monte Carlo tests. I'd like to share a few of the ones that I use most often:

- The **choice** function from the builtin random libary. This let's you choose one item from a sequence (e.g. a list) with each item getting equal chances of being drawn. More formally this is a uniform random choice.

In [268]:
#What shall I eat today?
breakfast_foods = ["Fig Oatmeal with Pecans","Nasi Goreng","Bagel with Cream Cheese",\
         "Moin-Moin and Akamu","Grits","Kaya Toast","Wentelteefjes","Cereal","Chechebsa","Northwest Scramble"]

#Honestly I *love* breakfast and if I got a chance to try all these I couldn't choose. Let's make python do it.

#Since the choice function is part of the random module 
#we have to import it before we can use it:
from random import choice

breakfast = choice(breakfast_foods)
print(f"Today's Breakfast is: {breakfast}")

Today's Breakfast is: Chechebsa


- The **choices** function is a fancy version of choice that allows you to put a 'weight' on each choice. The chance that an item is selected is proportional to its weight. You can also select more than one item at once by setting the k variable:


In [269]:
breakfast_foods = ["Fig Oatmeal with Pecans","Nasi Goreng","Bagel with Cream Cheese",\
         "Moin-Moin and Akamu","Grits","Kaya Toast","Wentelteefjes","Cereal","Chechebsa","Northwest Scramble"]

#Sadly my actual distribution of typical breakfast foods is not as exciting as the above:
weights = [0.80,0.0,0.10,0.0,0.0,0.0,0.0,0.0,0.0,0.10]
from random import choices

n_choices = 30
breakfast_this_month = choices(breakfast_foods,weights=weights,k=n_choices)

#Convert our list to a string using commas as the delimiter
breakfast_this_month = ", ".join(breakfast_this_month)

print(f"My breakfast for the next month is: {breakfast_this_month}")

My breakfast for the next month is: Fig Oatmeal with Pecans, Fig Oatmeal with Pecans, Fig Oatmeal with Pecans, Fig Oatmeal with Pecans, Fig Oatmeal with Pecans, Fig Oatmeal with Pecans, Fig Oatmeal with Pecans, Fig Oatmeal with Pecans, Fig Oatmeal with Pecans, Fig Oatmeal with Pecans, Fig Oatmeal with Pecans, Northwest Scramble, Fig Oatmeal with Pecans, Northwest Scramble, Fig Oatmeal with Pecans, Fig Oatmeal with Pecans, Fig Oatmeal with Pecans, Fig Oatmeal with Pecans, Fig Oatmeal with Pecans, Northwest Scramble, Fig Oatmeal with Pecans, Fig Oatmeal with Pecans, Northwest Scramble, Northwest Scramble, Fig Oatmeal with Pecans, Fig Oatmeal with Pecans, Fig Oatmeal with Pecans, Fig Oatmeal with Pecans, Fig Oatmeal with Pecans, Fig Oatmeal with Pecans


- The **shuffle** function in the random module rearranges the order of a list or other sequence. Note that as lists are mutable this rearrangement happens 'in place', meaning we are changing the original list rather than returning a new shuffled list. 

In [270]:
from random import shuffle
shuffle(breakfast_foods)
#Since the rearrangement happens 'in place ' we just write
#shuffle(breakfast foods) 
#not
#shuffled_foods = shuffle(breakfast_foods)


#Convert the first 7 breakfast items to a comma delimited list
this_weeks_menu = ", ".join(breakfast_foods[:7])
print(f"This week's menu is: {this_weeks_menu}")

This week's menu is: Chechebsa, Bagel with Cream Cheese, Northwest Scramble, Cereal, Kaya Toast, Fig Oatmeal with Pecans, Wentelteefjes


- The **random** function generates a [pseudorandom](https://en.wikipedia.org/wiki/Random_number_generation) number between 0 and 1. The result can be any floating point number greater than or equal to 0.0 but less than 1.0. The random function is often paired with an if statement to simulate an event with a particular probability.

Here's an example:

In [271]:
from random import random
chance_of_rain = 0.10
weather = "Sunny"
if random() <= 0.10:
    weather = "Rainy"
    
print(f"Today the weather is {weather}")

Today the weather is Sunny


## Using randomization to implement a simple Monte Carlo test 

Let's implement the simple Monte Carlo test from above about the chances of drawing 3 aces out of a 5 card hand drawn from a standard 52 card deck.  

In [272]:

#Since we only care about the aces, let's model our deck as 4 Aces + 48 other cards
deck = ["Ace of Spades","Ace of Clubs","Ace of Hearts","Ace of Diamonds"] + ["Other Card"] * 48

#We will need a function to randomize our list of cards
#the shuffle function from the random module will randomize 
#the order of a list in place
from random import shuffle

#Use variables to hold the number of trials
#our hand size and the number of aces needed for 'success'
#so they are easy to change
n_trials = 100000
hand_size = 5
n_aces_needed = 3

#Now try drawing 5 cards many times and count up successes.
# Each 'success' will be a hand in which we got 3 or more Aces.

successes = 0
for i in range(n_trials):
    #Shuffle the deck!
    shuffle(deck)
    
    #Draw a hand with hand_size cards in it
    hand = deck[:hand_size]
    
    #Count the number of cards with 'Ace' in their name
    n_aces_in_hand = 0
    for card in hand:
        if 'Ace' in card:
            n_aces_in_hand +=1
            
    #Test if we drew 3 or more aces,
    #if so, count this draw as a 'success'
    if n_aces_in_hand >= 3:
        successes +=1
        
#Done! Our probability is now just the 
#number of successes divided by the number of 
#times we tried.

p = successes / n_trials

print(f"The probability of drawing {n_aces_needed} aces in a hand of {hand_size} based on {n_trials} Monte Carlo trials: {p}")
print("Analytical answer:", 0.00173607904)        
    
    

The probability of drawing 3 aces in a hand of 5 based on 100000 Monte Carlo trials: 0.00195
Analytical answer: 0.00173607904


The analytical answer (as outlined [here](https://socratic.org/questions/what-is-the-probability-that-the-deal-of-a-five-card-hand-provides-at-least-one-), is based on the number of possible hands that have 3 aces (4,512) divided by the total number of possible hands (2,598,960). Compare that against our simulated answer. Rerun the code and do the same several times in a row. 

There are several things to notice here - 

First, the Monte Carlo p-value depends on random simulation, and will therefore generally not be the same each time you run the test - sometimes it will be a little bit too high, sometimes too low.

Second, the Monte Carlo simulation is more accurate the more trials you run. If you draw only 10 times (i.e. rerun setting the n_trials variable equal to 10), you may *never* see 3 aces in a hand, resulting in an incorrectly low p-value of 0.0.  However, as the number of trials increases, your Monte Carlo p-value should ever more closely approximate the real p-value.


#### Making our Monte Carlo code more modular

Now that we've got some code working, let's revise it to make it more modular. If we break down what the code is doing there are 3 main parts:

1. Simulating the situation (drawing 5 cards from a simulated deck)
2. Testing if our statistical event of interest occurred in the simulation (were there 3 or more aces in the hand?)
3. Code to run the simulations and tests for our condition of interest in a for loop and report the final p value

Each of these 3 parts can be turned into a python function. This potentially allows us to reuse the code in similar situations.

In [273]:

def simulate_draw(deck,n_cards):
    """Shuffle deck and return a hand of n cards
    n_cards --  the number of cards to draw as an int
    deck -- a list of cards (note deck will be shuffled in place)
    """
    shuffle(deck)
    hand = deck[:hand_size]
    return hand

def test_for_cards(hand,card_value = "Ace",n_needed=3):
    """Return True if enough cards are in hand, False otherwise"""
    n_cards_of_interest = 0
    for card_name in hand:
        if card_value in card_name:
            n_cards_of_interest +=1
    
    if n_cards_of_interest >= n_needed:
        return True
    
    return False

def monte_carlo_test(deck,card_value="Ace",n_needed=3,hand_size=5,n_trials=10000):
    """Return the probability of drawing a certain number of a particular card from a deck
    deck -- a deck represented by a list of python strings
    card_value -- the name of the card we want to draw (any card with this in the name will count)
    n_needed -- the number of the card that you have to draw in a given hand for it to count
    hand_size -- the number of cards to draw in each hand
    n_trials -- the number of times to run the simulation
    """
    successes = 0
    for trial in range(n_trials):
        hand = simulate_draw(deck,n_cards=hand_size)
        if test_for_cards(hand,card_value=card_value,n_needed=n_needed):
           successes += 1
    
    p = successes / n_trials
    return p

deck = ["Ace of Spades","Ace of Clubs","Ace of Hearts","Ace of Diamonds"] + ["Other Card"] * 48
monte_carlo_test(n_trials=10000,deck=deck,card_value="Ace",n_needed=3)

0.0021

## Biological Example: Tuatara Sex and Climate Change

To start thinking about Monte Carlo methods, it is useful to begin with a [motivating example](http://datanuggets.org/wp-content/uploads/2016/11/Tuatara-climate-change_StudentA.pdf). [Tuatara](https://en.wikipedia.org/wiki/Tuatara) are unique animals that live only in New Zealand. 

Like many animals, Tuatara's biological sex is not fixed by purely genetic factors, but rather can also be influenced strongly by their environment. In the case of Tuataras, the temperature at which eggs are incubated determines sex - with low temperatures tending to produce females and higher temperatures producing males.

Let us consider how we might test whether unequal Tuatara sex ratios could be due to chance. For example, in August of 2000, Dr. Kristine Grayson and colleagues ([Grayson *et al*., 2014](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0094214)) report capturing 34 Tuatara, of which ~67.6% (23/34) were male and ~32.4% (11/34) were female. We might be interested to know if this data supports the idea that the environmental conditions in August of 2000 were driving Tuatara to an unequal sex ratio. How can we test this quantitatively? We will try to do so using a few general steps that we can reuse on other Monte Carlo problems. Here are the steps:


- Write code for simulating the event we want to study
- Write code for calculating whether the statistical event we are interested in occurs in a simulation. 
- Write code for running many simulations, adding up how often our event of interest occurs, and computing a Monte Carlo p value.

#### State the statistical event we wish to quantify

Before we dive into writing code, we should take a moment to think about the statistical event we want our p-value to represent.

In frequentist statistics, the key to answering this question is to consider what null hypotheses we are testing. Here the main alternative hypothesis we've identified is that capturing 69.7%  male Tuatara's might be simply due to chance. Even if sex ratios were in reality equal we might have just gotten 'unlucky' in which 34 Tuatara were tagged. We want the p-value for getting 23/34 male Tuatara if the population is really 50% male (the null hypothesis). 

Before we go on, and in order to remind ourselves later when we are interpreting our p-value, it's useful to write down exactly what null hypothesis we are testing:

> *Null hypothesis*: the population of Tuatara is equally balanced between male and female Tuatara.

Now that we have that out of the way, what is our statistical event of interest? In frequentist statistical hypothesis testing, the event of interest is often the chances of observing an **effect size** as big or bigger than  the observed one under a *null hypothesis* that represents an uninteresting, trivial or random outcome. If it is common to see as big of a difference as the one we saw under the null hypothesis, then our data do not provide strong evidence against the null hypothesis, and we would say that they are 'not statistically significant'. If instead we find that it is extremely unlikely to see as big of an effect as we actually observed, *if* the null hypothesis were true, then this argues that the null hypothesis is not responsible for producing our effect, and the effect we see is therefore 'statistically significant'.

Of course, this only helps a little - we still have to say what we mean by **effect size**. Here, the effect we are interested in is the difference in the number of male vs. female Tuatara we caught. We could express our effect size in several ways. For example, we might measure our effect size as the raw difference in the number of male vs. female Tuatara, the Male:Female sex ratio we calculate, or in their proportion (67% male). For now let's go with the proportion male.

Like the null hypothesis it is useful to write down how we will calculate the effect size we are intersted in:
> *Effect size*: the percentage of male Tuatara in our sample. 

This then allows us to write down our statistical event of interest: 

> *Statistical Event of Interest*: how often do we see Male tuatara compose 69.7% or more of a sample of 34 Tuatara? 


#### Write code for a Monte Carlo test

Now that we have identified our statistical event of interest, and written some notes that will let us interpret the resulting p-value, we can write our code. 

The Monte Carlo test will end up using a very similar recipe to the one in our deck of cards. For now, we need to worry about writing code to do two things:

1. Simulating the situation (simulate catching 34 Tuatara *if* the null hypothesis were true and the population was 50% male and 50% female)
2. Testing if our statistical event of interest occurred in the simulation (were there 69.7% or more Males)
3. Write code to run the simulations and tests for our condition of interest in a for loop and report the final p value


If this number is less than 0.05 your result would traditionally be considered statistically significant, in the sense that your null hypothesis had a less than 5% chance of producing such an extreme outcome.

This can be a lot, but if we take it step by step and build simple functions hopefully the logic will be clear. Once you get it you can use this for a huge variety of statistical tests by modifying the code in steps 1 and 2.

First we'll simulate catching a single tuatara that might be male or female. To do this we'll write a function that let's us 'catch' a lizard and will return either the string 'Male' or 'Female'. 


In [274]:
#The random function will get us a number between 0.0 and 1.0 inclusive.
from random import random

# The catch tuatara function that simulates catching a single Tuatara and returns 'male' or 'female'
def catch_tuatara(proportion_male = 0.5):
    """Returns 'Male' or 'Female' based on catching a random Tuatara
    
    proportion_male -- the percentage of Tuatara that are male.
    
    Notes: All Tuatara are assumed to be male or female 
    """
    if random() < proportion_male:
        return 'Male'
    else:
        return 'Female'
    

Let's try out the function a few times just to make sure we understand what it's doing:

In [275]:
print("You catch a Tuatara! It's a", catch_tuatara())
print("You catch a Tuatara! It's a", catch_tuatara())
print("You catch a Tuatara! It's a", catch_tuatara())
print("You catch a Tuatara! It's a", catch_tuatara())
print("You catch a Tuatara! It's a", catch_tuatara())

You catch a Tuatara! It's a Male
You catch a Tuatara! It's a Male
You catch a Tuatara! It's a Female
You catch a Tuatara! It's a Male
You catch a Tuatara! It's a Male


So now we can simulate catching a single Tuatara. But the statistical event we really wanted to simulate was catching 34 Tuatara and having some number of them be male. So we need to use this code to write another function that simulates catching a number of Tuatara during an expedition, and returns to us their sexes. We can do this by running our code for catching a single Tuatara in a for loop.

In [276]:
#The random function will get us a number between 0.0 and 1.0 inclusive.
from random import random

#We need to simulate catching many Tuatara. 
#We do so using a for loop that repeatedly
#simulates catching individual Tuatara 

def catch_many_tuatara(n=34,proportion_male=0.5):
    """Return a list of the sex of Tuatara caught
    n -- the number of Tuatara caught
    proportion_male -- the simulated percentage of males in the population
    """
    result = []
    for i in range(n):
        
        #Simulating a single Tuatara
        if random() < proportion_male:
            sex = 'Male'
        else:
            sex ='Female'
        result.append(sex)
    return result

Let's try running this code a few times to be sure we understand what the output looks like. Note that each time the exact result will be a bit different (since the simulation invoves an element of randomness):

In [277]:
print("Here are the Tuatara you caught:",catch_many_tuatara())

Here are the Tuatara you caught: ['Female', 'Female', 'Female', 'Female', 'Male', 'Male', 'Male', 'Male', 'Female', 'Male', 'Female', 'Male', 'Female', 'Female', 'Male', 'Female', 'Male', 'Male', 'Female', 'Male', 'Female', 'Male', 'Female', 'Female', 'Female', 'Female', 'Male', 'Male', 'Female', 'Female', 'Female', 'Male', 'Male', 'Male']


This latest function is now giving us a simulation of the statistical event we wanted to test - catching many tuatara and having a certain number of them be male. So we can check Step 1 off of our list. We now need to take
the output shown above and write code to calculate an effect size based on it. We said our effect size would be measured as the number of 'excess' males - so we can calculate this just by counting up the number of males and females in our result, and then subtracting the number of females from the number of males. (Note that this will result in a negative number if there are fewer males than females and that's OK).

In [278]:
# Now that we have code to simulate an outcome, we need code to 
#calculate effect size based on that outcome
def calc_effect_size_percent_males(tuatara_captured):
    """Return the percentage of males captured 
    tuatara_captured -- a list of 'Male' or 'Female' for each tuatara captured
     example: ['Male','Female','Male']
    """
    
    n_male = 0
    n_female = 0
    for tuatara in tuatara_captured:
        if tuatara == 'Male':
            n_male +=1
        elif tuatara == 'Female':
            n_female +=1
        else:
            raise ValueError("Entries for each Tuatara should be 'Male' or 'Female'" +
              "(note capitalization matters). Entry %s is neither" %tuatara)
    
    return n_male / (n_male + n_female)

Let's try our code out with an example where we know what the right answer should be: if we catch 10 tuatara and 7 are male but 3 are female, we want our code to return 4, because there are 4 more male than female tuatara. Let's try it out:

In [279]:
tuatara_captured = ["Male","Male","Male","Male","Male","Male","Male","Female","Female","Female"]
print(calc_effect_size_excess_males(tuatara_captured))

4


Great! So as expected we can measure an effect size in terms of the number of excess males for each simulation. Now we have to write a final Monte Carlo function that repeatedly simulates lizard populations, counts excess males, and records when the number of excess males exceeeds what we saw in real life (the observed effect size).



In [280]:
def monte_carlo(simulation_fn,effect_size_fn,observed_effect_size,n_trials=10000):
    """Return the Monte Carlo p value, the simulated (null) effect size and the observed one
    
    simulation_fn -- a function to simulate the null hypothesis
    effect_size_fn -- a function to simulate the effect_size
    n_trials -- the number of trials to run. More trials will more precisely estimate p.
    
    """
    #The successes variable will ask how often the *simulated* effect size
    #(under the null hypothesis) equals or exceeds the *observed* effect size
    successes = 0
    for trial in range(n_trials):
        simulated_result = simulation_fn()
        effect_size = effect_size_fn(simulated_result)
        if effect_size >= observed_effect_size:
            successes +=1
    
    monte_carlo_p_value = successes/n_trials
    return monte_carlo_p_value   
 


Note that for reusability this Monte Carlo function takes as parameters the names of the functions we want to use for simulation and effect size. You pass these in just like any other parameter, but be careful to leave off the parentheses when passing the name of the function - we want to pass the function itself not the results of calling it. 

In [281]:
observed_effect_size = 0.697 # We saw 23/34 males, so by our measure the effect size is 69.7% or 0.697 
p = monte_carlo(simulation_fn = catch_many_tuatara,
  effect_size_fn = calc_effect_size_percent_males,
  observed_effect_size = observed_effect_size) 
print("Monte Carlo p value:",p)

Monte Carlo p value: 0.0117


The exact p-value you get back will vary a bit each time you run the code. Recall that this is because the Monte Carlo approach makes use of random simulation. I get a value of around 0.01. This suggests that we should expect to catch 69.7% or more male Tuatara out of 34 total only about 1% of the time, *if in truth there are equal numbers of male and female Tuatara*. This chance is quite low, suggesting that the null hypothesis - while still possible - is quite unlikely to explain our data.  

In frequentist statistics, a p-value of <= 0.05 - representing a 5% or lower chance of producing your effect size or larger under the null hypothesis - is often taken as a conventional threshold for statistical significance. Conversely if our final p-value were > 0.05, we would conclude that the difference in male vs. female Tuatara we caught was not statistically significant, since there was a fairly large chance it could have happened by chance.

It's worth noting that while p <= 0.05 is a traditional threshold for 'statistical significance', the p-value itself is more important than the threshold

If you did all that you have now implemented two Montel Carlo simulations in python! Huge congratulations! This is a really big step that will help you understand many aspects of statistics. Now that we've been through the process a couple times, we can use the same ideas to test a variety of different statistical hypothesis about our Tuatara. 

## Exercises
- **Modify the Monte Carlo simulation of card drawing to find the chances of drawing 3 Aces in a hand of 7 cards**. *Hint*: you shouldn't need to change more than a single parameter to test this.

- **Modify the Monte Carlo simulation of card drawing to find the chances of drawing 4 Aces in a hand of 5 cards**. *Hint*: you shouldn't need to change more than a single parameter to test this.

- **Modify the Monte Carlo simulation of card drawing to find the chances of drawing *no* Aces in a hand of 7 cards**. 

- **Modify the Monte Carlo simulation of sex ratio significance to account for unequal capture rates**.  When conducting our Monte Carlo test for an even sex ratio in Tuatara, we assumed that male and female Tuatara were captured at equal rates. [Herrel *et al.* 2010](https://academic.oup.com/biolinnean/article/100/2/287/2450663) measured the length and bite force of male and female Tuatara and found that male Tuatara on average weighted 434.8 g while female Tuatara on average weighed 272.7 g. Let us assume for a moment that the chances of observing a male vs. female Tuatara are proportional to their mass, with more massive Tuatara's being easier to spot. (The details of this assumption are probably unrealistic, but hopefully the overall idea will still be illustrative).   If our observation were correct, then a true 50:50 sex ratio of Tuatara might result in an *observed* sex ratio that included more males (because they are, under our assumption, easier to spot). Specifically we'd expect to observe about 61.5% males even if the true sex ratio were 50:50. Modify the calculation of significance for the figure from ([Grayson *et al*., 2014](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0094214)) to calculate significance if the above assumption were correct. 